In [1]:
from delierium.Infinitesimals import prolongationODE
from delierium.JanetBasis import Janet_Basis

In [4]:
from IPython.core.debugger import set_trace
function('xi phi')
def infinitesimalsODE (ode, dependent, independent):
    prolongation = prolongationODE(ode, dependent, independent)[0].expand()
    s1                 = solve(ode==0, diff(dependent(independent),independent,independent))
    determining_system = prolongation.subs({s1[0].lhs() : s1[0].rhs()}).simplify()
    # so far it is checked manually and with mathematica
    determining_system = \
        [_ [0] for _ in determining_system.coefficients(diff(dependent(independent), independent))]
    # remove hardcoded xi, phi
    for _ in determining_system:
        _.show()
    #set_trace()
    janet = Janet_Basis(determining_system, [phi, xi], [dependent, independent])
    return janet

In [5]:
x = var('x')
y = function("y")
ode1=4*diff(y(x),x,x)*y(x) - 3* diff(y(x),x)**2-12*y(x)**3
inf = infinitesimalsODE(ode1, y ,x)

12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - 24*phi(y(x), x)*y(x)^2 + 4*y(x)*D[1, 1](phi)(y(x), x)

-36*y(x)^3*D[0](xi)(y(x), x) + 8*y(x)*D[0, 1](phi)(y(x), x) - 4*y(x)*D[1, 1](xi)(y(x), x) - 6*D[1](phi)(y(x), x)

4*y(x)*D[0, 0](phi)(y(x), x) - 8*y(x)*D[0, 1](xi)(y(x), x) + 3*phi(y(x), x)/y(x) - 3*D[0](phi)(y(x), x)

-4*y(x)*D[0, 0](xi)(y(x), x) - 3*D[0](xi)(y(x), x)

In [6]:
for _ in inf.S:
    _.show()
    _.Lder().show()

-3/4*D[0](phi)(y(x), x)/y(x) + 3/4*phi(y(x), x)/y(x)^2 + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

D[0, 0](phi)(y(x), x)

-9/2*y(x)^2*D[0](xi)(y(x), x) - 3/4*D[1](phi)(y(x), x)/y(x) + D[0, 1](phi)(y(x), x) - 1/2*D[1, 1](xi)(y(x), x)

D[0, 1](phi)(y(x), x)

3*y(x)^2*D[0](phi)(y(x), x) - 6*y(x)^2*D[1](xi)(y(x), x) - 6*phi(y(x), x)*y(x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

3/4*D[0](xi)(y(x), x)/y(x) + D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [8]:
ode2=4*diff(y(x),x,x)*y(x)**2 - 3* (x**2)*diff(y(x),x)*2-12*y(x)**3
inf = infinitesimalsODE(ode2, y ,x)
print("=======")
for _ in inf.S:
    _.show()
    _.Lder().show()

12*y(x)^3*D[0](phi)(y(x), x) - 24*y(x)^3*D[1](xi)(y(x), x) - 12*phi(y(x), x)*y(x)^2 - 6*x^2*D[1](phi)(y(x), x) + 4*y(x)^2*D[1, 1](phi)(y(x), x)

-36*y(x)^3*D[0](xi)(y(x), x) + 8*y(x)^2*D[0, 1](phi)(y(x), x) - 6*x^2*D[1](xi)(y(x), x) - 4*y(x)^2*D[1, 1](xi)(y(x), x) - 12*x*xi(y(x), x) + 12*x^2*phi(y(x), x)/y(x)

4*y(x)^2*D[0, 0](phi)(y(x), x) - 12*x^2*D[0](xi)(y(x), x) - 8*y(x)^2*D[0, 1](xi)(y(x), x)

-4*y(x)^2*D[0, 0](xi)(y(x), x)

-3*x^2*D[0](xi)(y(x), x)/y(x)^2 + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

...

D[0, 1](phi)(y(x), x)

3*y(x)*D[0](phi)(y(x), x) - 6*y(x)*D[1](xi)(y(x), x) - 3/2*x^2*D[1](phi)(y(x), x)/y(x)^2 - 3*phi(y(x), x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [10]:
ode3=diff(y(x),x,x) + y(x)*diff(y(x),x)+x*y(x)**4
inf = infinitesimalsODE(ode3, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()

-x*y(x)^4*D[0](phi)(y(x), x) + 2*x*y(x)^4*D[1](xi)(y(x), x) + 4*x*phi(y(x), x)*y(x)^3 + xi(y(x), x)*y(x)^4 + y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

3*x*y(x)^4*D[0](xi)(y(x), x) + y(x)*D[1](xi)(y(x), x) + phi(y(x), x) + 2*D[0, 1](phi)(y(x), x) - D[1, 1](xi)(y(x), x)

2*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

-D[0, 0](xi)(y(x), x)

2*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

...

D[0, 1](phi)(y(x), x)

-x*y(x)^4*D[0](phi)(y(x), x) + 2*x*y(x)^4*D[1](xi)(y(x), x) + 4*x*phi(y(x), x)*y(x)^3 + xi(y(x), x)*y(x)^4 + y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [11]:
ode4=diff(y(x),x,x) + 3*y(x)*diff(y(x),x)+y(x)**3
inf = infinitesimalsODE(ode4, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()

-y(x)^3*D[0](phi)(y(x), x) + 2*y(x)^3*D[1](xi)(y(x), x) + 3*phi(y(x), x)*y(x)^2 + 3*y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

3*y(x)^3*D[0](xi)(y(x), x) + 3*y(x)*D[1](xi)(y(x), x) + 3*phi(y(x), x) + 2*D[0, 1](phi)(y(x), x) - D[1, 1](xi)(y(x), x)

6*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

-D[0, 0](xi)(y(x), x)

6*y(x)*D[0](xi)(y(x), x) + D[0, 0](phi)(y(x), x) - 2*D[0, 1](xi)(y(x), x)

...

D[0, 1](phi)(y(x), x)

-y(x)^3*D[0](phi)(y(x), x) + 2*y(x)^3*D[1](xi)(y(x), x) + 3*phi(y(x), x)*y(x)^2 + 3*y(x)*D[1](phi)(y(x), x) + D[1, 1](phi)(y(x), x)

D[1, 1](phi)(y(x), x)

D[0, 0](xi)(y(x), x)

D[0, 0](xi)(y(x), x)

In [12]:
ode5=diff(y(x),x,x,x) + y(x)*diff(y(x),x,x)
inf = infinitesimalsODE(ode5, y ,x)
print("=========")
for _ in inf.S:
    _.show()
    _.Lder().show()

y(x)*D[1, 1](phi)(y(x), x) - D[1](xi)(y(x), x)*diff(y(x), x, x, x) - phi(y(x), x)*diff(y(x), x, x, x)/y(x) - 3*D[0, 1](phi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[1, 1](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) - 3*D[0](xi)(y(x), x)*diff(y(x), x, x, x)^2/y(x)^2 + D[1, 1, 1](phi)(y(x), x)

2*y(x)*D[0, 1](phi)(y(x), x) - y(x)*D[1, 1](xi)(y(x), x) - D[0](xi)(y(x), x)*diff(y(x), x, x, x) - 3*D[0, 0](phi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 9*D[0, 1](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[0, 1, 1](phi)(y(x), x) - D[1, 1, 1](xi)(y(x), x)

y(x)*D[0, 0](phi)(y(x), x) - 2*y(x)*D[0, 1](xi)(y(x), x) + 6*D[0, 0](xi)(y(x), x)*diff(y(x), x, x, x)/y(x) + 3*D[0, 0, 1](phi)(y(x), x) - 3*D[0, 1, 1](xi)(y(x), x)

-y(x)*D[0, 0](xi)(y(x), x) + D[0, 0, 0](phi)(y(x), x) - 3*D[0, 0, 1](xi)(y(x), x)

-D[0, 0, 0](xi)(y(x), x)

...

D[0, 1, 1](phi)(y(x), x)

y(x)*D[1, 1](phi)(y(x), x) - 3*D[0](xi)(y(x), x)*diff(y(x), x, x, x)^2/y(x)^2 - phi(y(x), x)/y(x) - 3*D[0, 1](phi)(y(x), x)/y(x) + 3*D[1, 1](xi)(y(x), x)/y(x) + D[1, 1, 1](phi)(y(x), x) - D[1](xi)(y(x), x)

D[1, 1, 1](phi)(y(x), x)

D[0, 0, 0](xi)(y(x), x)

D[0, 0, 0](xi)(y(x), x)